In [2]:
import pandas as pd
df = pd.read_csv("./data/combined_ticks.csv", index_col=0)

In [3]:
prophet_df = df.reset_index(names="ds")
prophet_df

,ds,S&P500_Close,Crude Oil_Close,Silver_Close,Iron_Close,Natural Gas_Close,NYSE Composite_Close,Chinese Yuan_Close,Candian Dollar_Close,Euro_Close,VIX_Close,Interest Rate 10y_Close,Interest Rate 30y_Close,Gold_Close
0,2010-10-14,1173.810059,82.690002,24.417000,148.789993,3.657,7546.589844,0.150240,0.996910,1.396726,19.879999,2.495,3.898,1376.699951
1,2010-10-15,1176.189941,81.250000,24.271999,146.000000,3.535,7520.600098,0.150525,0.995718,1.407202,19.030001,2.576,3.999,1371.099976
2,2010-10-18,1184.709961,83.080002,24.396999,143.070007,3.431,7571.100098,0.150807,0.987264,1.396433,19.090000,2.491,3.927,1371.199951
3,2010-10-19,1165.900024,79.489998,23.764000,140.919998,3.513,7423.649902,0.150734,0.983158,1.393592,20.629999,2.475,3.902,1335.099976
4,2010-10-20,1178.170044,81.769997,23.848000,142.880005,3.539,7523.810059,0.150725,0.968054,1.373909,19.790001,2.470,3.888,1343.300049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3555,2024-11-29,6032.379883,68.000000,30.684999,102.440002,3.363,20272.039062,0.138066,0.713919,1.056301,13.510000,4.178,4.366,2657.000000
3556,2024-12-02,6047.149902,68.099998,30.448999,105.320000,3.213,20213.220703,0.138108,0.713643,1.054507,13.340000,4.196,4.361,2634.899902
3557,2024-12-03,6049.879883,69.940002,31.077000,106.080002,3.042,20185.810547,0.137535,0.712099,1.050122,13.300000,4.223,4.394,2644.699951
3558,2024-12-04,6086.490234,68.540001,31.504999,106.290001,3.043,20188.599609,0.137291,0.710939,1.051083,13.450000,4.180,4.347,2653.800049


In [4]:
from prophet import Prophet
import pandas as pd
import matplotlib.pyplot as plt

# Ensure 'ds' column is in datetime format
prophet_df['ds'] = pd.to_datetime(prophet_df['ds'])

# Define the target variable and regressors
target_column = 'Gold_Close'
regressors = [
    'S&P500_Close', 'Crude Oil_Close', 'Silver_Close', 'Iron_Close', 
    'Natural Gas_Close', 'NYSE Composite_Close', 'Chinese Yuan_Close',
    'Candian Dollar_Close', 'Euro_Close', 'VIX_Close', 
    'Interest Rate 10y_Close', 'Interest Rate 30y_Close'
]

# Rename the target column to 'y' for Prophet compatibility
prophet_df = prophet_df.rename(columns={target_column: 'y'})

# Split the data into training and test sets
train_size = int(len(prophet_df) * 0.8)
train_df = prophet_df[:train_size]
test_df = prophet_df[train_size:]

# Initialize the Prophet model
model = Prophet()

# Add each regressor to the model
for regressor in regressors:
    model.add_regressor(regressor)

# Fit the model on the training data
model.fit(train_df)

# Make predictions on the test data
forecast = model.predict(test_df)

Importing plotly failed. Interactive plots will not work.
16:01:42 - cmdstanpy - INFO - Chain [1] start processing
16:01:44 - cmdstanpy - INFO - Chain [1] done processing


In [5]:
final = pd.DataFrame({
    "Actual": test_df["y"].reset_index(drop=True),
    "Predicted": forecast["yhat"]
})
final = final[1:].reset_index(drop=True)

In [7]:
final.to_csv("prophet_comparison.csv")